In [271]:
from sympy import *
from sympy import Derivative as D, Function as F
from sympy.solvers.solveset import linsolve
from sympy.solvers.ode import ode_nth_linear_constant_coeff_homogeneous as ode_solve
from IPython.display import display

init_printing(use_latex='mathjax', wrap_line=False)
# TODO init_session?
# TODO shit, how to display stuff as single line???
from IPython.core.display import display, HTML
no_wrap_div = '<div style="white-space: nowrap">{}{}</div>'
# display(HTML(no_wrap_div.format(eqX, eqT)))

x, t = symbols('x t', real=True)
u = F('u')(x, t)
eq = Eq(D(u, x, 2) - D(u, t, 2))

print("PDE:")
display(eq)

X = F('X')(x)
T = F('T')(t)

k = symbols('k')
L = symbols('L', real=True)

# TODO sympy: can't handle without manual separation
E = k ** 2
[_eqX, _eqT] = pde_separate_mul(eq, u, [X, T])

display(_eqX)

eqX = Eq(_eqX - E) # ).canonical
eqT = Eq(_eqT - E) # ).canonical
# canonical moves all variables to the left and numbers to the right


print("Separating variables:")
display((eqX, eqT))


x_l = 0
x_r = 1
print("Boundary_conditions: ")
print("x_l:", x_l)
print("x_r:", x_r)

Tsol = dsolve(eqT).rhs
# shit, it won't eliminate solutions based on k :(
Xsol = dsolve(
    eqX,
    X,
    #ics={
    #    X.subs(x, x_l): 0,
    #    X.subs(x, x_r): 0,            
    #},
    hint='nth_linear_constant_coeff_homogeneous',
    # returns = 'both',
    
).rhs

print("General solutions for X and T:")
display((Xsol, Tsol))

# how to specify constants for DE solver??
[C1, C2] = list(Xsol.free_symbols.difference({k, x}))

bcs_l = Eq(Xsol.subs(x, x_l))
bcs_r = Eq(Xsol.subs(x, x_r))

print("BC enforces:")
display((bcs_l, bcs_r))


# TODO blog about that, give examples?
# so, solveset stuff can't handle 
[c1_c2] = solve(bcs_l, C1)
# solve(bcs_r.subs(C1, c1_c2))
# [{C2:0},{k:0},{k:iπ}]
# TODO what do we do with C2 = 0 solution???
K = solveset(bcs_r.subs(C1, c1_c2), k)

print("Allowed values of k:")
display(K)

# basis = imageset(lambda kk: Xsol.subs(k, kk), K)
# print("Solutions basis:")
# display(basis)

# here, sympy retuns something called ImageSet. it basically means that the solutions image of Z under x -> x i pi


# right, how to define infinite coefficients? can it just be a function?
# TODO how to sum something over imageset????

# TODO base_set??


# shit, that didn't work. requires lower and upper bounds
# kinda makes sense, we can't just sum over arbitrary sets...
# but we do want to sum over countable sets!

def sum_map(f, iset, idx):
    return Sum(
        f(idx, iset.lamda(idx)),
        (idx, iset.base_set._inf, iset.base_set._sup),
    )

A = IndexedBase('A')
B = IndexedBase('B')
n = symbols('n')

sol = sum_map(lambda idx, term: Xsol.subs({
    C1: A[idx],
    C2: B[idx],
    k : term,
}), K, idx=n)


print('General solution:')
display(sol)
# TODO do we need to eliminate negative coefficients??

# imageset(lambda u: u**2, allowed_ks)
#Xsol
# print(allowed_ks.domain)
# ok, now we basically restrict ourselves to considering only specific waves

# TODO pick first N coeffs and do initial transform?

#bcs = [
#    ,
#    Xsol.subs(x, x_r),
#]
#linsolve(bcs, [C1, C2], exclude=[k], simplify=False)
#bcs

# usol = Xsol.rhs * Tsol.rhs
# linsolve(, [C1, C2])

# TODO shit. it doesn't seem to work..
# classify_ode(eqX - k)
# Xsol
# classify_ode(eqX - k, X)

# dir(Xsol.subs(x, x_l))

# TODO ugh, c1 c2??


# usol
# TODO bcs

PDE:


    2              2             
   ∂              ∂              
- ───(u(x, t)) + ───(u(x, t)) = 0
    2              2             
  ∂t             ∂x              

  2      
 d       
───(X(x))
  2      
dx       
─────────
   X(x)  

Separating variables:


⎛         2                     2          ⎞
⎜        d                     d           ⎟
⎜       ───(X(x))             ───(T(t))    ⎟
⎜         2                     2          ⎟
⎜   2   dx                2   dt           ⎟
⎜- k  + ───────── = 0, - k  + ───────── = 0⎟
⎝          X(x)                  T(t)      ⎠

...

⎛                 -k       k    ⎞
⎝C₁ + C₂ = 0, C₁⋅ℯ   + C₂⋅ℯ  = 0⎠

Allowed values of k:


{n⋅ⅈ⋅π | n ∊ ℤ}

General solution:


  ∞                                    
 ___                                   
 ╲                                     
  ╲    ⎛ ⅈ⋅π⋅n⋅x         -ⅈ⋅π⋅n⋅x     ⎞
  ╱    ⎝ℯ       ⋅B[n] + ℯ        ⋅A[n]⎠
 ╱                                     
 ‾‾‾                                   
n = -∞                                 

In [139]:
# shit. it can't handle restrictions on k :(
from sympy.solvers.solveset import linsolve as slinsolve

A, B, k = symbols('A B k')


slinsolve([
    A * exp(sqrt(-k)) + exp(sqrt(k)),
], [A])

# right, it just can't figure out multiple solutions??
# solveset(exp(k) - 1)

# shit, it can't handle eliminating certain k's :( 
slinsolve([
    A,
    0,
    #exp(k) - 1,
], [A, B])

E = symbols('E', real=True)
k = symbols('k')

# TODO shit, it won't handle sqrt??
solveset(exp(k) - exp(-k))


# solveset(exp(k) - 1)
solveset(exp(sqrt(k**2)) - 1)

slinsolve([
    A + B,
    A * exp(- k) + B * exp(k),
],
    [A, B]
)


#solve([
#    exp(k) + 1,
#], returns='solveset')
